In [75]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import time
from scipy import stats
import networkx as nx
from networkx.algorithms import bipartite

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
expression_df = pd.read_table('./data/brem2005_RNA_expression.csv')
genotypes_df = pd.read_table('./data/jbloom_strains_genotyped.tab')

In [3]:
strain_data = pd.read_csv('./data/brem2005_strains.csv')
strain_names = strain_data['Strain name'].tolist()
progeny_names = strain_names[18:]

In [4]:
expression_df.head()

,ID_REF,IDENTIFIER,By1,By2,BY3,BY4,BY5,BY6,RM1_1,RM2_1,...,22_4_d,22_5_d,23_3_d,23_5_d,24_1_d,25_1_d,25_3_d,25_4_d,26_1_d,26_2_d
0,0,TRS120,-0.0620,-0.094,0.0925,-0.1135,-0.289,-0.0420,-0.0570,-0.0140,...,0.0615,0.0640,-0.4475,0.109,0.1445,0.3130,-0.0185,-0.0360,0.1210,0.0575
1,1,SCC2,0.1340,0.066,NaN,-0.0460,-0.409,-0.4100,0.4235,0.2045,...,0.0770,0.2280,0.3815,0.239,0.1045,0.3535,-0.0300,0.0775,0.4845,0.1150
2,2,FLO1,-0.1165,-0.209,NaN,-0.0050,0.078,NaN,0.8970,0.6460,...,0.0735,0.2610,-0.0545,0.074,0.2690,0.4370,0.2475,0.0840,-0.1570,0.0040
3,3,MYO3,-0.1595,-0.228,NaN,0.1830,-0.102,0.1815,-0.1555,0.0445,...,-0.0875,-0.0995,-0.5545,0.132,0.2220,0.2430,0.1830,-0.0040,-0.1085,0.2220
4,4,PDR10,0.0395,-0.001,NaN,0.0430,-0.162,0.1560,0.2075,0.1125,...,0.0360,0.1130,0.1615,0.011,0.2985,-0.0760,-0.2335,0.1015,-0.2010,-0.3375


In [5]:
genotypes_df[genotypes_df.columns.tolist()[0:128]].head()

,RQTL_name,2_7_a,2_7_b,2_7_c,2_7_d,21_5_c,22_1_d,22_2_d,19_2_c,19_3_c,...,9_7_d,26_2_d,19_1_c,21_3_d,21_1_d,15_6_c,17_1_a,name,chromosome,position
0,YAL069W_1,0,1,1,0,1,0,0,0,0,...,0,1,1,2,0,0,1,YAL069W,1,483
1,YAL069W_2,0,1,1,0,1,2,0,0,0,...,0,1,1,2,2,0,1,YAL069W,1,484
2,NAL013C_3,0,2,1,0,1,0,0,0,0,...,0,1,1,0,0,0,1,NAL013C,1,3220
3,NAL013C_4,0,2,1,0,1,0,0,0,0,...,0,1,1,0,0,0,1,NAL013C,1,3223
4,NAL013C_5,0,2,1,0,1,0,0,0,0,...,0,1,1,0,0,0,1,NAL013C,1,3232


In [6]:
# Divide all progeny into groups by their inheritance pattern
# for a given genetic marker, and then plot the data clouds
# to visually observe if there is any correlation between marker
# and RNA expression 


def expression_by_RNA_and_marker(RNA_name, marker_name):
    from_BY, from_RM = [], []
    RNA_pos = expression_df[expression_df.IDENTIFIER == RNA_name].index.tolist()[0]
    marker_pos = genotypes_df[genotypes_df.RQTL_name == marker_name].index.tolist()[0]
    for strain in progeny_names:
        expression_value = expression_df[strain].iloc[RNA_pos]
        inheritance_pattern = genotypes_df[strain].iloc[marker_pos]
        if inheritance_pattern == 0:
            from_BY.append(expression_value)
        elif inheritance_pattern == 1:
            from_RM.append(expression_value)
    return from_BY, from_RM


def test_linkage(RNA_name, marker_name, eps=1e-5):
    from_BY, from_RM = expression_by_RNA_and_marker(RNA_name, marker_name)
    statistic, pvalue = stats.mannwhitneyu(x=from_BY, y=from_RM)
    return (pvalue <= eps, pvalue)


def plot_expression_to_marker_correlation(RNA_name, marker_name):
    from_BY, from_RM = expression_by_RNA_and_marker(RNA_name, marker_name)
    xlabels = np.append(
                    np.full((1, len(from_BY)), 1), 
                    np.full((1, len(from_RM)), 2))\
                    + np.random.normal(0, 0.01, len(from_BY) + len(from_RM)
            )  
    ylabels = np.array(from_BY + from_RM)
    plt.figure(figsize=(20, 10))
    plt.rcParams["axes.facecolor"] = 'white'
    plt.title("p-value: {}".format(pvalue))
    plt.xlabel("class label")
    plt.ylabel("expression value")
    plt.scatter(
        x=xlabels, y=ylabels,
        c=ylabels, cmap=cm.jet
    )
    plt.savefig("./img/" + RNA_name + "_to_" + marker_name + ".png")
    plt.close()

**TODO**:
    1.  Find QTLs for every gene using 5% pvalue threshold
    2.  Validate QTLs using permutations test and calculate FDR
    3.  Construct and visualize a bipartite graph visualizing 
        the linkages found during experiments 
    4.  Also, it's worth calculating the linkages for every
        marker gene and plotting the bar chart, placing markers
        accordingly to their position in the chromosome.

In [69]:
# It's quite hard to identify QTLs just by looking at the point clouds.
# I guess, the next step is to apply MWU and then plot the pairs with 
# the highest predicted linkage, just to verify the validity of a program

# When and how to apply multiple-testing correction (FDR, q-value)?
# Also, it would be interesting to plot a graph of interconnections
# between markers and expressed genes 

np.random.seed(int(time.time()))
# Extract subsample
test_pairs = list(
                zip( 
                    np.random.choice(expression_df["IDENTIFIER"].tolist(), 1000, replace=True),
                    np.random.choice(genotypes_df["RQTL_name"].tolist(), 1000, replace=True)
                )
            )
# Check linkage for all entries of this subsample
# and construct a graph of interactions.
# Also, the minimal pvalue is also calculated
# in this code snippet, it's a temporary thing
min_pvalue = 1.
argmin_RNA, argmin_marker_name = "", ""
linkage_graph = nx.Graph()
for RNA_name, marker_name in test_pairs:
    if marker_name.count("Control") > 0:
        print("WTFWTFWTF")
    result, pvalue = test_linkage(RNA_name, marker_name, eps=0.05)
    if result:
        if not linkage_graph.has_node(RNA_name):
            linkage_graph.add_node(RNA_name, bipartite=0)
        if not linkage_graph.has_node(marker_name):
            linkage_graph.add_node(marker_name, bipartite=1)
        linkage_graph.add_edge(RNA_name, marker_name)
        if pvalue < min_pvalue:
            min_pvalue = pvalue
            argmin_RNA = RNA_name
            argmin_marker_name = marker_name
            # plot_expression_to_marker_correlation(RNA_name, marker_name)
print("{}: {} to {}".format(min_pvalue, argmin_RNA, argmin_marker_name))

6.659067656776818e-11: Phi174 Control to NKL015C_1698


In [105]:
# Here the linkage graph is analyzed and plotted
# It should be visualized as a bipartite graph,
# but it is not implemented yet.
maxdegree = 0
argmaxdegree = ""
for node in linkage_graph.nodes():
    d = linkage_graph.degree(node)
    if d > maxdegree:
        maxdegree = d
        argmaxdegree = str(node)
print(maxdegree, argmaxdegree)

# top_v, bottom_v = bipartite.sets(linkage_graph)
# doesn't work for some strange reasons
# I guess, I should either use DiGraph class
# or rewrite this as a separate function
top_v, bottom_v = [], []
for node, data in linkage_graph.nodes(data=True):
    if data["bipartite"] == 0:
        bottom_v.append(node)
    else:
        top_v.append(node)
# To plot a bipartite graph correctly, the positions
# of the vertices must be written down explicitly
pos = dict()
pos.update((n, (1, 2*i)) for i, n in enumerate(top_v))
pos.update((n, (2, 2*i)) for i, n in enumerate(bottom_v))

# Maybe, (20, 20) is too small: the labels overlap significantly
plt.figure(figsize=(20, 30))
nx.draw(
    linkage_graph,
    with_labels=True,
    node_size=50,
    edge_width=3.0,
    pos=pos,
    node_color=list(linkage_graph.degree().values()),
    edge_color='b',
    cmap=plt.cm.Blues,
    alpha=0.5
)
plt.savefig("./img/graph.png")
plt.close()

2 PKH2


/home/vvi/anaconda3/lib/python3.5/site-packages/networkx/drawing/nx_pylab.py:126: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  b = plt.ishold()
/home/vvi/anaconda3/lib/python3.5/site-packages/networkx/drawing/nx_pylab.py:138: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  plt.hold(b)
/home/vvi/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:917: UserWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
  warnings.warn(self.msg_depr_set % key)
/home/vvi/anaconda3/lib/python3.5/site-packages/matplotlib/rcsetup.py:152: UserWarning: axes.hold is deprecated, will be removed in 3.0
  warnings.warn("axes.hold is deprecated